In [1]:
import pandas as pd
import numpy as np
from pathlib2 import Path
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional
import re
import lightgbm as lgb
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score

In [2]:
def evraz_metric(true_t, predict_t, true_c, predict_c):

    delta_c = np.abs(np.array(true_c) - np.array(predict_c))
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(np.array(true_t) - np.array(predict_t))
    hit_rate_t = np.int64(delta_t < 20)

    N = np.size(predict_c)

    return np.sum(hit_rate_c + hit_rate_t) / 2 / N

In [3]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [4]:
path = Path('../../../data/2021_evraz')

In [5]:
target_train = pd.read_pickle(path.joinpath('target_train_wo_gas_wo_sip.pkl'))
print(target_train.shape)
target_train.head(3)

(2063, 145)


,NPLV,TST,C,VES,T,SI,MN,S,P,CR,...,plavka_TIPE_FUR_koniceskaa,plavka_TIPE_FUR_zilindriceskaa,plavka_TIPE_GOL_4-soplh54,plavka_TIPE_GOL_4-soplx54,plavka_TIPE_GOL_5soplovaa,plavka_TIPE_GOL_601-5,plavka_TIPE_GOL_E32,plavka_TIPE_GOL_E37_4-soplh54,plavka_TIPE_GOL_E_4-soplh54,pol_ras_sum
0,510008,1690,0.060,263700.0,1396.0,0.44,0.22,0.023,0.097,0.03,...,0,1,0,0,1,0,0,0,0,593045.405789
1,510009,1683,0.097,264500.0,1419.0,0.68,0.20,0.017,0.087,0.02,...,0,1,0,0,1,0,0,0,0,567680.040137
2,510010,1662,0.091,263800.0,1384.0,0.56,0.26,0.017,0.096,0.03,...,0,1,0,0,1,0,0,0,0,591476.261946


In [58]:
test = pd.read_pickle(path.joinpath('test_wo_gas_wo_sip.pkl'))
print(test.shape)
test.head(3)

(780, 143)


,NPLV,VES,T,SI,MN,S,P,CR,NI,CU,...,plavka_TIPE_FUR_koniceskaa,plavka_TIPE_FUR_zilindriceskaa,plavka_TIPE_GOL_4-soplh54,plavka_TIPE_GOL_4-soplx54,plavka_TIPE_GOL_5soplovaa,plavka_TIPE_GOL_601-5,plavka_TIPE_GOL_E32,plavka_TIPE_GOL_E37_4-soplh54,plavka_TIPE_GOL_E_4-soplh54,pol_ras_sum
0,512324,240100.0,1355.0,0.46,0.33,0.027,0.079,0.01,0.01,0.02,...,0,1,1,0,0,0,0,0,0,463015.154170
1,512327,266400.0,1390.0,0.30,0.33,0.032,0.099,0.01,0.00,0.00,...,0,1,1,0,0,0,0,0,0,499984.992640
2,512328,270200.0,1373.0,0.00,0.00,0.000,0.000,0.00,0.00,0.00,...,0,1,1,0,0,0,0,0,0,468358.082766


In [7]:
X_train, X_valid = train_test_split(
    target_train.drop(['TST', 'C', 'DATA_ZAMERA'], axis=1), train_size=0.75, shuffle=True, random_state=1)

y_train_t, y_valid_t = train_test_split(
    target_train['TST'], train_size=0.75, shuffle=True, random_state=1)

y_train_c, y_valid_c = train_test_split(
    target_train['C'], train_size=0.75, shuffle=True, random_state=1)



X_valid, X_test = train_test_split(
    X_valid, train_size=0.75, shuffle=True, random_state=1)

y_valid_t, y_test_t = train_test_split(
    y_valid_t, train_size=0.75, shuffle=True, random_state=1)

y_valid_c, y_test_c = train_test_split(
    y_valid_c, train_size=0.75, shuffle=True, random_state=1)



y_train_true_c, y_valid_true_c = train_test_split(
    target_train['C'], train_size=0.75, shuffle=True, random_state=1)
y_valid_true_c, y_test_true_c = train_test_split(
    y_valid_true_c, train_size=0.75, shuffle=True, random_state=1)

y_train_true_t, y_valid_true_t = train_test_split(
    target_train['TST'], train_size=0.75, shuffle=True, random_state=1)
y_valid_true_t, y_test_true_t = train_test_split(
    y_valid_true_t, train_size=0.75, shuffle=True, random_state=1)

print("x_train.shape = {} rows, {} cols".format(*X_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*X_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*X_test.shape))

x_train.shape = 1547 rows, 142 cols
x_valid.shape = 387 rows, 142 cols
x_test.shape = 129 rows, 142 cols


In [8]:
model_t = xgb.XGBRegressor(random_state=1)
model_t.fit(X_train, y_train_t)

model_c = xgb.XGBRegressor(random_state=1)
model_c.fit(X_train, y_train_c)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
train_score_t = r2_score(y_train_t, model_t.predict(X_train))
valid_score_t = r2_score(y_valid_t, model_t.predict(X_valid))
test_score_t = r2_score(y_test_t, model_t.predict(X_test))

train_score_c = r2_score(y_train_c, model_c.predict(X_train))
valid_score_c = r2_score(y_valid_c, model_c.predict(X_valid))
test_score_c = r2_score(y_test_c, model_c.predict(X_test))


print(f"Train-score: {round(train_score_t, 3)}, Valid-score: {round(valid_score_t, 3)}, Test-score: {round(test_score_t, 3)}")
print(f"Train-score: {round(train_score_c, 3)}, Valid-score: {round(valid_score_c, 3)}, Test-score: {round(test_score_c, 3)}")
print('Train metric = ', evraz_metric(y_train_c, model_c.predict(X_train), y_train_t, model_t.predict(X_train)))
print('Valid metric = ', evraz_metric(y_valid_c, model_c.predict(X_valid), y_valid_t, model_t.predict(X_valid)))
print('Test metric = ', evraz_metric(y_test_c, model_c.predict(X_test), y_test_t, model_t.predict(X_test)))

Train-score: 0.996, Valid-score: 0.418, Test-score: 0.561
Train-score: 0.999, Valid-score: 0.651, Test-score: 0.431
Train metric =  0.504524886877828
Valid metric =  0.5
Test metric =  0.5


In [10]:
np.random.seed(27)
scores_t = create_bootstrap_metrics(y_test_t, model_t.predict(X_test), r2_score)
scores_c = create_bootstrap_metrics(y_test_c, model_c.predict(X_test), r2_score)

print('Границы доверительных интервалов:')
print('t =', calculate_confidence_interval(scores_t))
print('c =', calculate_confidence_interval(scores_c))

Границы доверительных интервалов:
t = (0.42884865736877165, 0.6603542146315508)
c = (0.029864104197975336, 0.6416302194275111)


Получается, что по тесту у нас метрика качества 0.561 и 0.431. А когда мы оценили доверительный интервал, у нас получилось  t = (0.428, 0.660) и c = (0.029, 0.641). Это оцень широкий доверительный интервал. Он говорит, что равновероятно у нас метрика может иметь значение внутри этих границ. Т.е. наша валидация неустойчива.

Считается, что если ширина доверительного интервала больше 10%, то это не надёжный доверительный интервал.

In [11]:
model_t = xgb.XGBRegressor(random_state=1)
model_c = xgb.XGBRegressor(random_state=1)

cv_t = cross_val_score(
    estimator=model_t,
    X=target_train.drop(['TST', 'C', 'DATA_ZAMERA'], axis=1),
    y=target_train['TST'],
    scoring="r2",
    cv=5
)
cv_c = cross_val_score(
    estimator=model_c,
    X=target_train.drop(['TST', 'C', 'DATA_ZAMERA'], axis=1),
    y=target_train['C'],
    scoring="r2",
    cv=5
)

print(f"CV-results T: {round(np.mean(cv_t), 4)} +/- {round(np.std(cv_t), 3)}")
print(f"CV-results C: {round(np.mean(cv_c), 4)} +/- {round(np.std(cv_c), 3)}")

CV-results T: 0.42 +/- 0.132
CV-results C: 0.5413 +/- 0.109


In [56]:
def make_cross_validation(X: pd.DataFrame,
                          y_1: pd.Series,
                          y_2: pd.Series,
                          X_test: pd.DataFrame,
                          estimator_1: object,
                          estimator_2: object,
                          metric: callable,
                          cv_strategy,
                          error_to_be_outlier: None):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y_1: pd.Series
        Вектор 1-й целевой переменной.
        
    y_2: pd.Series
        Вектор 2-й целевой переменной.
        
    X_test: pd.Series
        Матрица признаков для предсказания.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.
        
    error_to_be_outlier: float, optional, default = None
        Максимальная относительная величина ошибки для того,
        чтобы объект считать выбросом и не учитывать в итоговой
        ошибке алгоритма. Опциональный параметр, по умолчанию,
        не используется.
        Если ставим 100, это 100% - если 2 раза ошибаемся, то
        говорим, что это выброс.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores_1, fold_train_scores_2, fold_valid_scores_1, fold_valid_scores_2, \
    evraz_metric_train_scores, evraz_metric_valid_scores = [], [], [], [], [], [], []
    oof_predictions_1, oof_predictions_2 = np.zeros(X.shape[0]), np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y_1)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train_1, y_valid_1 = y_1.loc[train_idx], y_1.loc[valid_idx]
        y_train_2, y_valid_2 = y_2.loc[train_idx], y_2.loc[valid_idx]

        estimator_1.fit(x_train, y_train_1)
        estimator_2.fit(x_train, y_train_2)
        y_train_pred_1 = estimator_1.predict(x_train)
        y_train_pred_2 = estimator_2.predict(x_train)
        y_valid_pred_1 = estimator_1.predict(x_valid)
        y_valid_pred_2 = estimator_2.predict(x_valid)

        fold_train_scores_1.append(metric(y_train_1, y_train_pred_1))
        fold_train_scores_2.append(metric(y_train_2, y_train_pred_2))
        
        if not error_to_be_outlier:
            fold_valid_scores_1.append(metric(y_valid_1, y_valid_pred_1))
            fold_valid_scores_2.append(metric(y_valid_2, y_valid_pred_2))
        else:
            mask = ((y_valid_1 - y_valid_pred_1) / y_valid_1) < error_to_be_outlier
            fold_valid_scores_1.append(metric(y_valid_1.loc[mask], y_valid_pred_1[mask]))
            mask = ((y_valid_2 - y_valid_pred_2) / y_valid_2) < error_to_be_outlier
            fold_valid_scores_2.append(metric(y_valid_2.loc[mask], y_valid_pred_2[mask]))
            
        oof_predictions_1[valid_idx] = y_valid_pred_1
        oof_predictions_2[valid_idx] = y_valid_pred_2
        
        evraz_metric_train = evraz_metric(y_train_1, y_train_pred_1, y_train_2, y_train_pred_2)
        evraz_metric_valid = evraz_metric(y_valid_1, y_valid_pred_1, y_valid_2, y_valid_pred_2)
        evraz_metric_train_scores.append(evraz_metric_train)
        evraz_metric_valid_scores.append(evraz_metric_valid)

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score 1 = {round(fold_train_scores_1[fold_number], 4)}, "
            f"train-score 2 = {round(fold_train_scores_2[fold_number], 4)}\n"
            f"valid-score 1 = {round(fold_valid_scores_1[fold_number], 4)}, "
            f"valid-score 2 = {round(fold_valid_scores_2[fold_number], 4)}" 
        )
        print(msg)
        print('Train metric = ', round(evraz_metric_train, 4))
        print('Valid metric = ', round(evraz_metric_valid, 4))
        print("="*69)
        estimators.append(estimator_1)
        estimators.append(estimator_2)

    if not error_to_be_outlier:
        oof_score_1 = metric(y_1, oof_predictions_1)
        oof_score_2 = metric(y_2, oof_predictions_2)
    else:
        mask = ((y_1 - oof_predictions_1) / y_1) < error_to_be_outlier
        oof_score_1 = metric(y_1.loc[mask], oof_predictions_1[mask])
        mask = ((y_2 - oof_predictions_2) / y_1) < error_to_be_outlier
        oof_score_2 = metric(y_2.loc[mask], oof_predictions_2[mask])
        
    print(f"CV-results train 1: {round(np.mean(fold_train_scores_1), 4)} +/- {round(np.std(fold_train_scores_1), 3)}")
    print(f"CV-results train 2: {round(np.mean(fold_train_scores_2), 4)} +/- {round(np.std(fold_train_scores_2), 3)}")
    print(f"CV-results valid 1: {round(np.mean(fold_valid_scores_1), 4)} +/- {round(np.std(fold_valid_scores_1), 3)}")
    print(f"CV-results valid 2: {round(np.mean(fold_valid_scores_2), 4)} +/- {round(np.std(fold_valid_scores_2), 3)}")
    print(f"OOF-score 1 = {round(oof_score_1, 4)}")
    print(f"OOF-score 2 = {round(oof_score_2, 4)}")
    
    print(f"CV-results evraz metric train: {round(np.mean(evraz_metric_train_scores), 4)}\
+/- {round(np.std(evraz_metric_train_scores), 3)}")
    print(f"CV-results evraz metric valid: {round(np.mean(evraz_metric_valid_scores), 4)}\
+/- {round(np.std(evraz_metric_valid_scores), 3)}")
    
    # error fix: ValueError: Feature shape mismatch, expected: 142, got 780
    # https://stackoverflow.com/questions/42338972/valueerror-feature-names-mismatch-in-xgboost-in-the-predict-function
    f_names = estimator_1.get_booster().feature_names 
    X_test = X_test[f_names]
    
    return estimators, oof_score_1, oof_score_2, fold_train_scores_1, fold_train_scores_2, fold_valid_scores_1,\
fold_valid_scores_2, oof_predictions_1, oof_predictions_2, estimator_1.predict(X_test), estimator_2.predict(X_test)

In [63]:
cv_strategy = KFold(n_splits=5)
#cv_strategy = StratifiedKFold(n_splits=5)

estimators, oof_score_1, oof_score_2, fold_train_scores_1, fold_train_scores_2, fold_valid_scores_1, \
fold_valid_scores_2, oof_predictions_1, oof_predictions_2, test_pred_1, test_pred_2 = make_cross_validation(
    target_train.drop(['TST', 'C', 'DATA_ZAMERA'], axis=1), 
    target_train['TST'], 
    target_train['C'],
    test.drop(['DATA_ZAMERA'], axis=1),
    estimator_1=xgb.XGBRegressor(random_state=1), 
    estimator_2=xgb.XGBRegressor(random_state=1),
    metric=r2_score, 
    cv_strategy=cv_strategy,
    error_to_be_outlier=None,
)

Fold: 1, train-observations = 1650, valid-observations = 413
train-score 1 = 0.9934, train-score 2 = 0.9982
valid-score 1 = 0.3857, valid-score 2 = 0.4216
Train metric =  0.9988
Valid metric =  0.6065
Fold: 2, train-observations = 1650, valid-observations = 413
train-score 1 = 0.995, train-score 2 = 0.9987
valid-score 1 = 0.4462, valid-score 2 = 0.4835
Train metric =  1.0
Valid metric =  0.6743
Fold: 3, train-observations = 1650, valid-observations = 413
train-score 1 = 0.9933, train-score 2 = 0.9988
valid-score 1 = 0.4912, valid-score 2 = 0.6232
Train metric =  1.0
Valid metric =  0.5932
Fold: 4, train-observations = 1651, valid-observations = 412
train-score 1 = 0.9945, train-score 2 = 0.9989
valid-score 1 = 0.5857, valid-score 2 = 0.4646
Train metric =  1.0
Valid metric =  0.6408
Fold: 5, train-observations = 1651, valid-observations = 412
train-score 1 = 0.9948, train-score 2 = 0.9989
valid-score 1 = 0.1913, valid-score 2 = 0.7135
Train metric =  1.0
Valid metric =  0.6201
CV-resul

In [67]:
submit = pd.concat([test['NPLV'], 
                    pd.Series(test_pred_1).rename('TST'), 
                    pd.Series(test_pred_2).rename('C')
                   ],
                   axis=1
                  )
print(submit.shape)
submit.head(5)

(780, 3)


,NPLV,TST,C
0,512324,1665.416870,0.032003
1,512327,1670.857300,0.039726
2,512328,1647.624268,0.036868
3,512331,1637.005127,0.025788
4,512333,1663.708618,0.042087


In [68]:
# получаем текущие дату и время
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# создаём путь и имя файла с датой и временем
# file_name = f'../../data/kaggle/gb_competitive_data_analysis/lgb_predictions_{now}.csv'
file_name = f'xgb_outlier_none_{now}.csv'
print('File name: ', file_name)

# сохраняем в csv
submit.to_csv(file_name, index=False, encoding='utf-8')
print('\n File saved to disk!')

File name:  xgb_outlier_none_2021-10-31_05-03-45.csv

 File saved to disk!
